This notebook aims to calculate the mean power levels for a specified date-range and save them.

The mean power is required to calculate the ACC (annomaly correlation coef).

In [ ]:
import sys
sys.path.append("/hkfs/home/project/hk-project-test-mlperf/om1434/masterarbeit")
sys.path.append("/home/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion")
sys.path.append("/home/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion/pangu_pytorch")
from wind_fusion.pangu_pytorch.finetune.finetune_power import create_dataloader
from wind_fusion.pangu_pytorch.era5_data.config import cfg
from datetime import datetime
import torch
from wind_fusion.pangu_pytorch.era5_data import utils_data

48h frequency for 2018 (Test-Data)

In [ ]:
test_dataloader = create_dataloader(
        cfg.PG.TEST.START_TIME,
        cfg.PG.TEST.END_TIME,
        cfg.PG.TEST.FREQUENCY,
        cfg.PG.TEST.BATCH_SIZE,
        False,
    )

In [ ]:
# Initialize accumulators
total_sum = None  # To store cumulative sum of `target_power_test`
total_samples = 0  # To store total number of samples

# Iterate through the test dataloader
for id, data in enumerate(test_dataloader, 0):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] Predicting on batch {id}")
    
    (
        input_test,
        input_surface_test,
        target_power_test,
        target_upper_test,
        target_surface_test,
        periods_test,
    ) = data

    # Sum the target_power_test along the batch axis
    batch_sum = target_power_test.sum(dim=0)
    
    # Update total sum
    if total_sum is None:
        total_sum = batch_sum
    else:
        total_sum += batch_sum

    # Update sample count
    total_samples += target_power_test.shape[0]  # Batch size

# Calculate mean
mean_per_grid_point = total_sum / total_samples
print("Mean per grid point calculated.")


In [ ]:
import matplotlib.pyplot as plt

# Assuming mean_per_grid_point is a 2D tensor
plt.imshow(mean_per_grid_point.squeeze(), cmap='viridis')
plt.colorbar()
plt.title('Mean Per Grid Point')
plt.show()

Save mean wind power

In [ ]:
import numpy as np
def save_tensor_as_npy(tensor, file_path):
    # Convert the torch tensor to a numpy array
    numpy_array = tensor.numpy()
    # Save the numpy array as a .npy file
    np.save(file_path, numpy_array)

def load_tensor_from_npy(file_path):
    # Load the numpy array from the .npy file
    numpy_array = np.load(file_path)
    # Convert the numpy array to a torch tensor
    tensor = torch.from_numpy(numpy_array)
    return tensor


# Assuming mean_per_grid_point is already defined as a torch tensor
save_path = "/home/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion/pangu_pytorch/data/aux_data/mean_power_per_grid_point.npy"
save_tensor_as_npy(mean_per_grid_point, save_path)

In [ ]:
test = load_tensor_from_npy(save_path)

In [ ]:
plt.imshow(test.squeeze(), cmap='viridis')

In [ ]:
aux_constants = utils_data.loadAllConstants(device="cpu")

surface_mean, _, upper_mean, _ = aux_constants["weather_statistics_last"]

In [ ]:
output_tensor = torch.load("/home/hk-project-test-mlperf/om1434/masterarbeit/wind_fusion/pangu_pytorch/result/Test/model_output/target_('2016042812',).pth", map_location="cpu")

plt.imshow(output_tensor.squeeze().detach().numpy(), cmap='viridis')

In [ ]:
weather_anomaly = output_tensor - mean_per_grid_point

In [ ]:
plt.imshow(weather_anomaly.squeeze().detach().numpy(), cmap='viridis')
plt.colorbar()